In [1]:
include("../../src/pure/Pnjl_pure.jl")

Trho (generic function with 1 method)

In [ ]:
T= 300.0
mu_B = 0.0
X0 = [-0.01, -0.01, -0.20, 0.8, 0.8]
phi = X0[1:3]
Phi1 = X0[4]
Phi2 = X0[5]
res = Omega(phi, Phi1, Phi2, T/hc, mu_B/hc,)